# Matching Compositions to Recordings

In [1]:
import pickle
import re

import numpy as np
import pandas as pd

### Bringing in Track + Composition Tables

In [9]:
track_list = pd.read_csv('../data/main_wfeats.csv', index_col=0)
comp_artists = pd.read_csv('../data/comp_artists.csv', index_col=0)
compositions = pd.read_csv('../data/compositions.csv', index_col=0)
artist_comp_lookup = pd.read_csv('../data/artist_comp_lookup.csv', index_col=0)

#### Combining Composition Tables

In [11]:
compositions.head()

,CID,AID,Title
0,0,360318916,FOR THA LOVE OF MONEY
1,1,530659306,WE THE PEOPLE
2,2,334030418,CELERY-TIME
3,3,442081954,NEUTRON BOMB
4,4,230055482,WILL THE CIRCLE BE UNBROKEN


In [12]:
artist_comp_lookup.head()

,CID,PID
0,0,0
1,1933,0
2,17624,0
3,17630,0
4,17633,0


In [13]:
comp_artists.head()

,Performer Name,PID
0,BONE,0
1,BONE THUGS N HARMONY,1
2,BONE THUGS N HARMONY FEAT. EAZY-E,2
3,BONE THUGS-N-HARMONY,3
4,BONE THUGS-N-HARMONY (EDITED),4


In [17]:
full_comps = pd.merge(artist_comp_lookup, compositions, on='CID')
full_comps = pd.merge(full_comps, comp_artists, on='PID')

In [18]:
full_comps.head()

,CID,PID,AID,Title,Performer Name
0,0,0,360318916,FOR THA LOVE OF MONEY,BONE
1,1933,0,350208616,ETERNAL,BONE
2,17624,0,340371584,DA INTRODUCTION,BONE
3,17630,0,350209419,EAST 1999,BONE
4,17633,0,350208465,EVERYDAY THANG,BONE


#### Examining Track Table

In [20]:
track_list.head()

,song_id,album_release_date,artist_id,artist_name,duration_ms,explicit,linked_album,song_title,danceability,energy,...,pv_dim_3,pv_dim_4,pv_dim_5,pv_dim_6,pv_dim_7,pv_dim_8,pv_dim_9,pv_dim_10,pv_dim_11,pv_dim_12
0,6SluaPiV04KOaRTOIScoff,1995-10-13,6UE7nl9mha6s8z0wFQFIZ2,Robyn,229226.0,False,Robyn Is Here,Show Me Love - Radio Version,0.546,0.643,...,0.231588,0.227392,0.365724,0.220462,0.367808,0.267055,0.344281,0.349016,0.323426,0.480299
1,5qEVq3ZEGr0Got441lueWS,2018-08-10,6S58b0fr8TkWrEHOH4tRVu,Switchfoot,247240.0,False,You Found Me (Unbroken: Path To Redemption),You Found Me (Unbroken: Path To Redemption),0.603,0.802,...,0.384941,0.397085,0.465443,0.237421,0.359981,0.209631,0.283483,0.188632,0.212271,0.490470
2,5kqIPrATaCc2LqxVWzQGbk,2016-04-01,25u4wHJWxCA9vO0CzxAbK7,Lukas Graham,237300.0,False,Lukas Graham,7 Years,0.765,0.473,...,0.341671,0.321183,0.195459,0.330539,0.175221,0.328568,0.153059,0.221073,0.444818,0.203276
3,3aVyHFxRkf8lSjhWdJ68AW,2013-01-01,0C0XlULifJtAgn6ZNCW2eu,The Killers,262000.0,False,Direct Hits,Just Another Girl,0.547,0.779,...,0.229995,0.264792,0.180531,0.281061,0.355194,0.189039,0.256742,0.193406,0.253140,0.308046
4,0zIyxS6QxZogHOpGkI6IZH,2018-09-07,0le01dl1WllSHhjEXRl4in,Tamia,236545.0,False,Passion Like Fire,Deeper,0.438,0.288,...,0.233717,0.128174,0.321370,0.200370,0.391387,0.132925,0.265942,0.537358,0.158429,0.266790


#### Standardizing Composition and Track Tables